# 1. 데이터 탐색

먼저, 전체 뉴스가 몇 개인지 세어보자.

In [ ]:
cand_list = ['문재인', '홍준표', '안철수', '유승민', '심상정']

In [ ]:
import json
import os

aid_list = []
aid_set = set()

for cand in cand_list:
    dir_path = os.path.join('./data/', cand)
    for file_name in os.listdir(dir_path):
        file_path = os.path.join(dir_path, file_name)
        with open(file_path, 'r', encoding='utf-8') as f:
            d = json.load(f)
        _, _, aid = file_name.replace('.json', '').split('-')
        aid_list.append(aid)

aid_set.update(aid_list) # aid_set = set(aid_list)

In [ ]:
print(len(aid_list), len(aid_set))

적어도 두 후보를 언급한 뉴스가 있다!

중복된 뉴스, 즉, 두 명 이상 언급한 뉴스는 제하고, 한 명만 언급한 뉴스만 남기기로 하자.

In [ ]:
from collections import Counter

dup_aid_list = [item for item, count in Counter(aid_list).items() if count > 1]
print(len(dup_aid_list))

uniq_aid_list = [item for item, count in Counter(aid_list).items() if count == 1]
print(len(uniq_aid_list))

이제 원활한 탐색과 전처리를 위해 pandas를 사용해 df라는 이름의 dataframe에 뉴스 데이터를 가져오자.

In [ ]:
import pandas as pd

index_list = uniq_aid_list
column_list = [
    'sid', 'oid', 'aid', 'url', 'title', 
    'contents', 'press', 'datetime', 'label'
]

df = pd.DataFrame(columns=column_list, index=uniq_aid_list)

In [ ]:
import json
import os

for cand in cand_list:
    dir_path = os.path.join('./data/', cand)
    for file_name in os.listdir(dir_path):
        file_path = os.path.join(dir_path, file_name)
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        sid, oid, aid = file_name.replace('.json', '').split('-')
        if aid in df.index.values:
            data['sid'], data['oid'], data['aid'] = sid, oid, aid
            data['label'] = cand
            for key in df.columns.values:
                df.loc[aid,key] = data[key]

`df`라는 DataFrame(데이터테이블)이 만들어졌다!

In [ ]:
df.info()

In [ ]:
df.sample(5)

# 뉴스의 길이, 뉴스 문장 개수

먼저, 이에 대한 column을 만들어보자.

In [ ]:
def get_num_sentence(s):
    sent_list = s.split('.')
    return len(sent_list)

df['num_sentence'] = df['contents'].apply(get_num_sentence)

def get_len(s):
    return len(s)

df['len_news'] = df['contents'].apply(get_len)

In [ ]:
df.head()

분포를 살펴보기 위해 시각화를 해보자.

In [ ]:
% matplotlib inline
import matplotlib

matplotlib.style.use('ggplot')

In [ ]:
df['len_news'].hist(bins=100)

In [ ]:
len(df[df.len_news > 5000])

In [ ]:
df['num_sentence'].hist(bins=100)

In [ ]:
len(df[df.num_sentence > 200])

In [ ]:
len(df[df.num_sentence <= 5])

In [ ]:
df = df[df.num_sentence > 5]

# count: 뉴스 개수 세어보기

by
- 후보
- 언론사
- 날짜
- 후보 & 날짜
- 후보 & 언론사
- 언론사 & 날짜
- 후보 & 언론사 & 날짜


### 후보별 count

In [ ]:
df.groupby(['label']).size().reset_index(name='count')

### 언론사별 count

In [ ]:
df.groupby(['press']).size().reset_index(name='count').sort_values(by=['count'], ascending=False)

### 언론사, 후보별 count

In [ ]:
tmp = df.groupby(['press', 'label']).size().reset_index(name='count')
tmp

In [ ]:
tmp = tmp.pivot(index='press', columns='label', values='count')
tmp

In [ ]:
cand = '안철수'
tmp.sort_values(by=[cand], ascending=False)

### 날짜별 count

`date` column을 만들어보자.

In [ ]:
def get_date(row):
    return row.datetime[:10]

# pd.options.mode.chained_assignment = None  # default='warn'
# http://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas

df['date'] = df.apply(get_date, axis=1)

In [ ]:
df.head()

In [ ]:
df.groupby(['date']).size().reset_index(name='count')

### 날짜, 후보별 count

In [ ]:
tmp = df.groupby(['date', 'label']).size().reset_index(name='count')
tmp

In [ ]:
tmp = tmp.pivot(index='date', columns='label', values='count')
tmp

In [ ]:
tmp.plot()

tmp.columns = ['mji', 'ssj', 'acs', 'ysm', 'hjp']
tmp.plot()

# 명사 추출하여 개수 세어보기

konlpy를 설치하고 사용해보자.

In [ ]:
import konlpy

In [ ]:
from konlpy.tag import Twitter
twitter = Twitter()

from konlpy.tag import Hannanum
hannanum = Hannanum()

from konlpy.tag import Kkma
kkma = Kkma()

누가 제일 잘하나 테스트해보자.

In [ ]:
x = df['contents'].iloc[1]
x

In [ ]:
twitter.nouns(x)

In [ ]:
kkma.nouns(x)

In [ ]:
hannanum.nouns(x)

twitter를 사용하기로 하자.

In [ ]:
noun_list = []
for c in df['contents']:
    noun_list.append(twitter.nouns(c))

In [ ]:
from collections import Counter
from itertools import chain

noun_list_flat = tuple(chain(*noun_list))
# print(noun_list_flat[:3])
count_dict = Counter(noun_list_flat)
count_dict.most_common(30)

In [ ]:
count_list = list(count_dict.items())
count_list.sort(key=lambda x: x[1], reverse=True)

wdf = pd.DataFrame(count_list, columns=['word', 'count'])
max_cut = 1000
wdf.hist(bins=100, range=(0, max_cut))
wdf[wdf['count'] > max_cut].sum()['count'] / wdf.sum()['count']

# 상위 N개의 단어를 사용하기로 하고, 뉴스 내 단어 등장여부를 나타내는 N개의 column을 만들어보자.

In [ ]:
for word, _ in count_dict.most_common(10):
    for index, row in df.iterrows():
        df.loc[index, word] = 1 if word in row['contents'] else 0

In [ ]:
df.head()